In [ ]:
from datasets import load_dataset
dataset = load_dataset("sled-umich/Action-Effect")

Extracting data files:   0%|          | 0/2 [00:00<?, ?it/s]

Generating ActionEffect split: 0 examples [00:00, ? examples/s]

In [ ]:
pip install datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 521.2/521.2 kB 8.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 16.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 20.1 MB/s eta 0:00:00


In [ ]:
import random
import base64
import requests
from PIL import Image
from io import BytesIO
import re
import numpy as np
import torch
import os
import json

In [ ]:
def seed_everything(seed_value):
    random.seed(seed_value)
    np.random.seed(seed_value)
    torch.manual_seed(seed_value)
    os.environ['PYTHONHASHSEED'] = str(seed_value)

    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed_value)
        torch.cuda.manual_seed_all(seed_value)
        torch.backends.cudnn.deterministic = True
        torch.backends.cudnn.benchmark = True

seed_everything(595)

In [ ]:
positive_images = dataset['ActionEffect']['positive_image_list']
verb_noun = dataset['ActionEffect']['verb noun']
all_images = []

for i in range(len(positive_images)):
    label = verb_noun[i]
    for image in positive_images[i]:
        all_images.append((label, image))

all_images = random.sample(all_images, 20)

In [ ]:
### One Shot Prompt
def get_openai_responses_few_shots(api_key, initial_image_files, image_files):

    initial_image_data_list = []

    for image_file in initial_image_files:
        buffer = BytesIO()
        image_file.save(buffer, format="JPEG")
        image_data = buffer.getvalue()
        base64_image = base64.b64encode(image_data).decode('utf-8')
        initial_image_data_list.append({
            "type": "image_url",
            "image_url": {
                "url": f"data:image/jpeg;base64,{base64_image}"
            }
        })

    image_data_list = []

    for image_file in image_files:
        buffer = BytesIO()
        image_file.save(buffer, format="JPEG")
        image_data = buffer.getvalue()
        base64_image = base64.b64encode(image_data).decode('utf-8')
        image_data_list.append({
            "type": "image_url",
            "image_url": {
                "url": f"data:image/jpeg;base64,{base64_image}"
            }
        })

    all_image_data_list = initial_image_data_list + image_data_list

    headers = {
        "Content-Type": "application/json",
        "Authorization": f"Bearer {api_key}"
    }

    # Prepare the payload for the request with multiple images
    payload = {
        "model": "gpt-4-vision-preview",  # Update this model name as per the latest API
        "messages": [
            {
                "role": "user",
                "content": [
                    {
                        "type": "text",
                        "text":  """ We are conducting an exercise to test the ability of an AI model to predict verb-noun pairs that describe the physical actions depicted in a series of images.\
                         We will upload a total of 11 images. \
                         The first image is training example and the answer is :\
                          [Shows an image of pitching a baseball] -> Correct Answer: 'pitch baseball'\
                        Your tasks should satisfy the following 3 conditions":
                        1.For the next 10 images, the task is to identify verb-noun pairs that best describe the actions taking place in each image. \
                        2.The model should generate 20 possible verb-noun pairs from the given list for each of the next 10 images, resulting in a total of 200 pairs.\
                        3.10 possible verb-noun pairs for each image shoule be different. \
                        Each pair should be formatted like 'arrange chairs', and separated by a comma and without including line breaks and in the below format:\
                         Image 1: bind hair, twist hair, label box, arrange flowers, arrange chairs, tie shoe, trim hair, carry bag, break glass, shave hair\
                         Image 2: like Image 1
                         ...
                         Image 10: like Image 1
                        4.The possible answers for the verb noun pairs are in this list: ['arrange chairs,arrange flowers,bake potato,beat eggs,\
                        bend knee,bend tree,bind hair,bite apple,block door,block window,boil egg,boil potato,break bowl,break cup,break door,break egg,break glass,\
                        break window,burn book,burn paper,burn tree,burn wood,burst balloon,burst door,carry bag,carry book,carry umbrella,chop carrot,chop meat,\
                        chop onion,chop tree,chop wood,close book,close cabinet,close door,close drawer,close window,coil rope,cook egg,cook meat,cook onion,\
                        cook potato,crack bottle,crack egg,crack glass,crack window,crash car,crop hair,cut apple,cut meat,cut onion,cut potato,cut tree,cut wood,\
                        fasten door,fasten window,fold paper,fry egg,fry meat,fry potato,grate carrot,grate potato,grind meat,hang bag,hang shirt,ignite paper,\
                        ignite wood,insert key,kick door,kick football,knot rope,label bottle,label box,lock cabinet,lock door,lock drawer,lock window,mash potato,\
                        mix eggs,open bottle,open box,open cabinet,open door,open drawer,open umbrella,open window,park car,peel apple,peel banana,peel carrot,\
                        peel orange,peel potato,pile books,pile boxes,pile wood,pitch baseball,ride bicycle,rip paper,roll paper,roll umbrella,saw tree,saw wood,\
                        scratch car,scratch knee,shave hair,shut door,shut window,skin knee,slice apple,slice meat,slice onion,slice potato,smash door,smash window,\
                        soak shirt,soak hair,spill coffee,split tree,split wood,squeeze orange,squeeze bottle,stain paper,stain shirt,stir coffee,stir soup,\
                        strip tree,tear book,tear shirt,tear paper,throw apple,throw baseball,throw football,throw frisbee,tie shoe,trim hair,trim tree,twist hair,\
                        twist rope,wrap book,wrap box']
                         """

                    },
                    *all_image_data_list
                ]
            }
        ],
        "max_tokens": 1000  #1000 is the best so far
    }

    # Send the request
    response = requests.post("https://api.openai.com/v1/chat/completions", headers=headers, json=payload).json()
    return response



### Experiments with 50 samples firstly

In [ ]:
positive_images = dataset['ActionEffect']['positive_image_list']
verb_noun = dataset['ActionEffect']['verb noun']
all_images = []
for i in range(len(positive_images)):
    label = verb_noun[i]
    for image in positive_images[i]:
        all_images.append((label, image))

all_images = random.sample(all_images,60)
images_50 = []
for item in all_images[1:51]:
    images_50.append(item[1])

images_50_label = []
for item in all_images[1:51]:
    images_50_label.append(item[0])

In [ ]:
# Convert to RGB
from PIL import Image
images_rgb = []
count_notrgb = 0
for image in images_50:
    if image.mode != 'RGB':
        image_rgb = image.convert('RGB')
        images_rgb.append(image_rgb)
        count_notrgb += 1
    else:
        images_rgb.append(image)
count_notrgb

1

In [ ]:
len(images_rgb)

50

In [ ]:
responses = []
api_key = "YOUR_OPENAI_KEY"  # Replace with your actual API key

for i in range(0, len(images_rgb), 10):
    images_10 = images_rgb[i:i+10]
    response_verb_noun = get_openai_responses_few_shots(api_key, initial_images, images_10)

    # Inspect the response
    print(response_verb_noun)

    try:
        content = response_verb_noun['choices'][0]['message']['content']
        # matches = re.findall(r"\n\d+\.\s*([a-zA-Z]+\s[a-zA-Z]+)[,\n]", content)
        # processed_data = [matches[i:i + 10] for i in range(0, len(matches), 10)]
        # for item in processed_data:
        responses.append(content)
    except KeyError:
        print("KeyError encountered. Check the response structure.")


{'id': 'chatcmpl-8PYia29Ozemxbr3JHPpSi1PO4pI31', 'object': 'chat.completion', 'created': 1701102348, 'model': 'gpt-4-1106-vision-preview', 'usage': {'prompt_tokens': 5603, 'completion_tokens': 650, 'total_tokens': 6253}, 'choices': [{'message': {'role': 'assistant', 'content': 'Image 1: bind hair, twist hair, label box, arrange flowers, arrange chairs, tie shoe, trim hair, carry bag, break glass, shave hair, roll paper, open umbrella, lock door, grate potato, fry potato, fold paper, close window, chop wood, boil potato, bake potato\n\nImage 2: close cabinet, lock cabinet, open cabinet, fasten door, lock door, open door, shut door, close door, lock drawer, open drawer, close drawer, shut window, close window, lock window, open window, fasten window, block door, block window, label bottle, pile boxes\n\nImage 3: ignite paper, burn paper, burn wood, ignite wood, throw frisbee, roll paper, twist rope, spill coffee, stain shirt, tie shoe, light fire, tear paper, break glass, crack bottle, o

In [ ]:
images_50_label[30:41]

['chop carrot',
 'ride bicycle',
 'pile books',
 'label bottle',
 'boil potato',
 'tear paper',
 'close door',
 'saw wood',
 'burn wood',
 'open cabinet',
 'ride bicycle']

In [ ]:
# Some data proecessing after running GPT4
final_result = []
for item in responses:
   matches1 = re.findall(r"Image \d+: ([\w\s,]+?)(?=\n)", item)
   if "Image 10:" in item:
    matches2 = re.findall(r"Image 10: ([\w\s,]+)", item)
   elif "Image 11:" in item:
    matches2 = re.findall(r"Image 11: ([\w\s,]+)", item)
   else:
    matches2 = []
   matches = matches1 + matches2
   print(matches[0])
   matches_flatten = []
   for pairs in matches:
    temp = pairs.split(', ')
    matches_flatten.append(temp)
   for data in matches_flatten:
    final_result.append(data)
len(final_result)

bind hair, twist hair, label box, arrange flowers, arrange chairs, tie shoe, trim hair, carry bag, break glass, shave hair, roll paper, open umbrella, lock door, grate potato, fry potato, fold paper, close window, chop wood, boil potato, bake potato
open umbrella, carry umbrella, hold umbrella, lift umbrella, display umbrella, handle umbrella, grip umbrella, present umbrella, use umbrella, extend umbrella, wave umbrella, brandish umbrella, clutch umbrella, wield umbrella, offer umbrella, raise umbrella, show umbrella, adjust umbrella, maneuver umbrella, position umbrella
crack egg, open bottle, spill coffee, break egg, grate potato, peel banana, crack bottle, peel orange, label bottle, roll paper, tie shoe, bind hair, bend knee, carry bag, boil egg, burn paper, arrange chairs, twist rope, chop carrot, mash potato
chop carrot, slice carrot, peel carrot, grate carrot, cut carrot, mash potato, slice potato, cut potato, grate potato, chop potato, label bottle, label box, tear paper, arrang

50

In [ ]:
# Accuracy of 50 samples
images_50_label = []
for item in all_images[1:51]:
    images_50_label.append(item[0])
correct_count = 0
for index, answer in enumerate(images_50_label):
    if answer in final_result[index]:
        correct_count += 1
accuracy = correct_count / len(images_50_label)
print(f'Accuracy: {accuracy * 100:.2f}%')

Accuracy: 94.00%


In [ ]:
len(final_result[0])

20

In [ ]:
len(final_result)

50

#### Experiment of 500 samples secondly

In [ ]:
positive_images = dataset['ActionEffect']['positive_image_list']
verb_noun = dataset['ActionEffect']['verb noun']
all_images = []
for i in range(len(positive_images)):
    label = verb_noun[i]
    for image in positive_images[i]:
        all_images.append((label, image))
random.seed(595)
# Sample 500 pictures from 2048 pictures
all_images = random.sample(all_images,501)
images_500 = []
for item in all_images[1:501]:
    images_500.append(item[1])
# Answers from the 500 samples
images_500_label = []
for item in all_images[1:501]:
    images_500_label.append(item[0])

few_images = all_images[:1]
initial_images = []
for item in few_images[:1]:
    initial_images.append(item[1])

In [ ]:
# Convert all pictures to RGB format
from PIL import Image
images_rgb_500 = []
count_notrgb = 0
for image in images_500:
    if image.mode != 'RGB':
        image_rgb = image.convert('RGB')
        images_rgb_500.append(image_rgb)
        count_notrgb += 1
    else:
        images_rgb_500.append(image)
count_notrgb

11

In [ ]:
# Divide 500 picturs into 10 chunks
splitted_lists = {}
for i in range(10):
    start_index = i * 50
    end_index = start_index + 50
    sublist = images_rgb_500[start_index:end_index]
    list_name = f"list_{i+1}"
    splitted_lists[list_name] = sublist

In [ ]:
# Run Chatgpt4
def run_model(image_list):
  responses500 = []
  # Replace with your actual API key
  api_key = "YOUR_OPENAI_KEY"

  for i in range(0, len(image_list), 10):
      images_10 = image_list[i:i+10]
      response_verb_noun = get_openai_responses_few_shots(api_key, initial_images, images_10)

      # Inspect the response
      print(response_verb_noun)

      try:
          content = response_verb_noun['choices'][0]['message']['content']
          responses500.append(content)
      except KeyError:
          print("KeyError encountered. Check the response structure.")
  return responses500

In [ ]:
result_list1 = run_model(splitted_lists['list_1'])

{'id': 'chatcmpl-8PfXdYJ270MpUDlAmiVWT2vb7bYZ1', 'object': 'chat.completion', 'created': 1701128577, 'model': 'gpt-4-1106-vision-preview', 'usage': {'prompt_tokens': 7793, 'completion_tokens': 712, 'total_tokens': 8505}, 'choices': [{'message': {'role': 'assistant', 'content': 'Image 1: crash car, park car, scratch car, open door, close door, lock door, fasten door, kick door, smash door, throw football, burst door, roll paper, fold paper, rip paper, tear paper, stain shirt, soak shirt, hang shirt, peel orange, throw frisbee\n\nImage 2: crash car, park car, scratch car, open door, close door, lock door, fasten door, kick door, smash door, throw football, burst door, roll paper, fold paper, rip paper, tear paper, stain shirt, soak shirt, hang shirt, peel orange, throw frisbee\n\nImage 3: open window, close window, break window, smash window, fasten window, lock window, block window, peel orange, throw frisbee, stain shirt, soak shirt, hang shirt, rip paper, fold paper, roll paper, throw

In [ ]:
import json
with open('result_list1.json', 'w') as file:
    json.dump(result_list1, file)

In [ ]:
result_list2 = run_model(splitted_lists['list_2'])

{'id': 'chatcmpl-8PfdyMBtpkdruenxjoRqhxQGMZKIg', 'object': 'chat.completion', 'created': 1701128970, 'model': 'gpt-4-1106-vision-preview', 'usage': {'prompt_tokens': 6773, 'completion_tokens': 646, 'total_tokens': 7419}, 'choices': [{'message': {'role': 'assistant', 'content': 'Image 1: coil rope, tie shoe, bind hair, twist rope, fasten window, lock cabinet, wrap box, arrange flowers, arrange chairs, chop wood, label box, carry bag, park car, pitch baseball, ride bicycle, throw frisbee, open umbrella, peel orange, hang shirt, saw tree\n\nImage 2: peel apple, slice apple, cut apple, arrange flowers, tie shoe, open bottle, label box, bend knee, bake potato, cook potato, mash potato, grate carrot, peel banana, peel carrot, chop onion, slice onion, cut onion, twist hair, ride bicycle, open door\n\nImage 3: cook soup, stir soup, mix eggs, peel carrot, peel potato, mash potato, cook potato, cook meat, chop onion, chop carrot, grate carrot, grate potato, slice meat, slice onion, slice potato,

In [ ]:
with open('result_list2.json', 'w') as file:
    json.dump(result_list2, file)

In [ ]:
result_list3 = run_model(splitted_lists['list_3'])

{'id': 'chatcmpl-8PhmouMnAXSJHbv17FCB8EULf3RgX', 'object': 'chat.completion', 'created': 1701137206, 'model': 'gpt-4-1106-vision-preview', 'usage': {'prompt_tokens': 5243, 'completion_tokens': 793, 'total_tokens': 6036}, 'choices': [{'message': {'role': 'assistant', 'content': 'Based on the images provided and the list of possible verb-noun pairs, here are the best matches for each image following the exercise\'s requirements:\n\nImage 1: pitch baseball, throw baseball, twist rope, bend knee, coil rope, crack bottle, fasten window, kick football, ride bicycle, saw wood, arrange flowers, break window, burst balloon, carry bag, chop tree, fold paper, ignite paper, peel banana, rip paper, stain shirt\n\nImage 2: stain shirt, spill coffee, burst door, smash door, break bowl, break glass, break cup, burn book, burn paper, tear paper, carry bag, carry book, coil rope, crop hair, fold paper, fry meat, grate potato, mash potato, open cabinet, ride bicycle\n\nImage 3: pitch baseball, throw base

In [ ]:
print(result_list3[4])

Image 1: pitch baseball, throw baseball, bend knee, carry bag, twist rope, label box, bind hair, close door, break glass, open door, park car, ride bicycle, roll paper, stir coffee, tie shoe, burn wood, chop wood, cut tree, arrange chairs, shut window

Image 2: ignite paper, ignite wood, burn tree, burn book, burn wood, carry bag, carry book, arrange chairs, arrange flowers, close drawer, close window, cut tree, fold paper, kick door, label bottle, mash potato, peel banana, roll umbrella, shut door, throw football

Image 3: cut potato, chop carrot, chop wood, slice potato, peel potato, chop meat, cook potato, fry potato, mash potato, grate potato, boil egg, boil potato, bind hair, burst balloon, carry bag, close cabinet, arrange chairs, pile wood, saw wood, stir soup

Image 4: crack glass, break glass, smash window, throw baseball, arrange flowers, bind hair, carry bag, chop wood, close drawer, fold paper, ignite wood, label bottle, open door, peel banana, pitch baseball, roll paper, s

In [ ]:
with open('result_list3.json', 'w') as file:
    json.dump(result_list3, file)

In [ ]:
result_list4 = run_model(splitted_lists['list_4'])

{'id': 'chatcmpl-8PfnfNG11nZAvekwmdlo9cQ1f8daw', 'object': 'chat.completion', 'created': 1701129571, 'model': 'gpt-4-1106-vision-preview', 'usage': {'prompt_tokens': 5243, 'completion_tokens': 649, 'total_tokens': 5892}, 'choices': [{'message': {'role': 'assistant', 'content': 'Image 1: rip paper, tear paper, roll paper, label box, open box, wrap box, block door, throw frisbee, stain paper, bind hair, twist hair, pitch baseball, chop wood, ride bicycle, throw apple, twist rope, wrap book, burst balloon, arrange chairs, arrange flowers\n\nImage 2: chop onion, cut onion, slice onion, cook onion, peel onion, grate carrot, throw apple, label bottle, arrange flowers, bend knee, pitch baseball, ride bicycle, chop wood, twist hair, bind hair, pile books, squeeze bottle, block window, boil potato, bake potato\n\nImage 3: cook meat, fry meat, chop onion, chop carrot, stir soup, stir coffee, cut meat, slice potato, chop meat, peel carrot, mix eggs, grate carrot, throw frisbee, pitch baseball, bi

In [ ]:
with open('result_list4.json', 'w') as file:
    json.dump(result_list4, file)

In [ ]:
result_list5 = run_model(splitted_lists['list_5'])

{'id': 'chatcmpl-8PiTHTQgifUiUSZ9v4OWAtaaCWl3c', 'object': 'chat.completion', 'created': 1701139839, 'model': 'gpt-4-1106-vision-preview', 'usage': {'prompt_tokens': 7283, 'completion_tokens': 710, 'total_tokens': 7993}, 'choices': [{'message': {'role': 'assistant', 'content': 'Image 1: pitch baseball, throw baseball, bend knee, twist rope, coil rope, lock cabinet, peel banana, squeeze bottle, roll paper, fold paper, tear book, ignite wood, bind hair, knot rope, chop carrot, cut apple, hang shirt, fry meat, ride bicycle, arrange chairs\n\nImage 2: bite apple, peel apple, slice apple, cut apple, throw apple, arrange flowers, bind hair, chop carrot, break glass, burst balloon, mash potato, fry potato, bake potato, boil potato, grate carrot, stain shirt, roll paper, fold paper, label box, carry bag\n\nImage 3: twist hair, bind hair, arrange flowers, tie shoe, label box, open bottle, peel banana, fasten door, shut door, bind hair, coil rope, hang bag, tear paper, fry meat, boil egg, mash p

In [ ]:
print(result_list5[4])

Image 1: pitch baseball, throw baseball, twist rope, bend knee, burst door, bind hair, roll paper, chop meat, carry umbrella, lock cabinet, smash window, open door, tie shoe, arrange flowers, rip paper, kick door, cut meat, shut door, spill coffee, catch baseball

Image 2: chop meat, cut meat, slice meat, arrange chairs, trim hair, label bottle, open box, peel potato, shut window, carry bag, cook meat, break egg, mash potato, fasten door, pile books, bend knee, lock cabinet, burn wood, break glass, twist hair

Image 3: fasten door, lock door, block door, shut door, close door, chain door, lock cabinet, bind hair, close cabinet, lock drawer, fasten window, block window, twist rope, coil rope, knot rope, tie shoe, roll paper, bend knee, trim hair, pile boxes

Image 4: wrap box, open box, peel banana, fold paper, label box, close drawer, arrange chairs, lock drawer, bind hair, trim hair, slice apple, cut potato, tear paper, pile books, shut window, close book, bake potato, pile wood, tie 

In [ ]:
with open('result_list5.json', 'w') as file:
    json.dump(result_list5, file)

In [ ]:
result_list6 = run_model(splitted_lists['list_6'])

{'id': 'chatcmpl-8PgKxqVF1CdpxWVKCTpYjccIch18A', 'object': 'chat.completion', 'created': 1701131635, 'model': 'gpt-4-1106-vision-preview', 'usage': {'prompt_tokens': 6263, 'completion_tokens': 648, 'total_tokens': 6911}, 'choices': [{'message': {'role': 'assistant', 'content': 'Image 1: lock window, close window, fasten window, open window, shut window, block window, smash window, crack window, break window, arrange flowers, tie shoe, bake potato, bend knee, bite apple, boil egg, chop carrot, close book, fold paper, fry egg, grate carrot\n\nImage 2: open box, label box, wrap box, arrange chairs, bind hair, bend tree, break glass, carry umbrella, chop meat, close drawer, coil rope, cook potato, crack bottle, crash car, cut apple, fasten door, fold paper, hang shirt, ignite paper, kick door\n\nImage 3: coil rope, pile boxes, tie shoe, label bottle, open drawer, break bowl, cut potato, fry meat, grate potato, hang bag, lock cabinet, mash potato, park car, peel banana, rip paper, saw wood,

In [ ]:
with open('result_list6.json', 'w') as file:
    json.dump(result_list6, file)

In [ ]:
result_list7 = run_model(splitted_lists['list_7'])

{'id': 'chatcmpl-8Pg82bvlq2oxGxoWGbLTNG13jn4fk', 'object': 'chat.completion', 'created': 1701130834, 'model': 'gpt-4-1106-vision-preview', 'usage': {'prompt_tokens': 5923, 'completion_tokens': 717, 'total_tokens': 6640}, 'choices': [{'message': {'role': 'assistant', 'content': "Image 1: pitch baseball, throw baseball, carry bag, wear cap, bend knee, tie shoe, wear glove, tuck shirt, bend arm, hold ball, grip ball, lift arm, wear uniform, stand mound, balance body, twist body, wear socks, clench teeth, squint eyes, aim ball\n\nImage 2: fry egg, cook egg, flip egg, serve egg, heat egg, season egg, crack egg, melt butter, use pan, prepare breakfast, spoil egg, plate egg, brown egg, overcook egg, make breakfast, garnish egg, center yolk, poach egg, oil pan, simmer egg\n\nImage 3: carry bag, hold sunglasses, wear watch, wear ring, flex muscles, wear bracelet, grip handle, show fashion, match accessories, wear stripes, dress stylishly, accessorize outfit, model handbag, pose wall, exhibit st

In [ ]:
with open('result_list7.json', 'w') as file:
    json.dump(result_list7, file)

In [ ]:
result_list8 = run_model(splitted_lists['list_8'])

{'id': 'chatcmpl-8PgD8k5kJmWbxb4QQTokVo0m3oSXh', 'object': 'chat.completion', 'created': 1701131150, 'model': 'gpt-4-1106-vision-preview', 'usage': {'prompt_tokens': 6603, 'completion_tokens': 642, 'total_tokens': 7245}, 'choices': [{'message': {'role': 'assistant', 'content': 'Image 1: arrange chairs, pile chairs, open drawer, close drawer, arrange flowers, open door, close door, lock drawer, fasten door, shut window, pile boxes, fold paper, park car, crack window, arrange chairs, tie shoe, break bowl, bind hair, bake potato, peel apple\n\nImage 2: spill coffee, pour coffee, break cup, stain shirt, soak shirt, open drawer, close drawer, peel banana, bend knee, break glass, spill coffee, open bottle, break bowl, sprinkle coffee, twist rope, pile books, peel carrot, twist hair, shut window, park car\n\nImage 3: slice apple, peel apple, cut apple, arrange flowers, twist rope, knot rope, label bottle, open drawer, bind hair, break egg, slice onion, chop carrot, close drawer, close door, p

In [ ]:
with open('result_list8.json', 'w') as file:
    json.dump(result_list8, file)

In [ ]:
result_list9 = run_model(splitted_lists['list_9'])

{'id': 'chatcmpl-8PgNFMuDKYxVXWE4S37Lu6c2oMuXk', 'object': 'chat.completion', 'created': 1701131777, 'model': 'gpt-4-1106-vision-preview', 'usage': {'prompt_tokens': 6773, 'completion_tokens': 715, 'total_tokens': 7488}, 'choices': [{'message': {'role': 'assistant', 'content': 'Image 1: pitch baseball\nImage 2: crack glass, break glass, smash window, crack window, break window, throw baseball, throw frisbee, throw apple, throw football, crash car, break bowl, break door, break cup, break egg, smash door, peel banana, peel carrot, peel orange, peel potato, peel apple\nImage 3: mash potato, boil potato, cook potato, bake potato, fry potato, grate potato, peel potato, cut potato, chop potato, slice potato, break egg, crack egg, beat eggs, mix eggs, fry egg, cook egg, peel banana, cut meat, chop meat, cook meat\nImage 4: wrap book, pile books, open box, label box, arrange flowers, bind hair, tear book, open bottle, close book, open door, close cabinet, close drawer, close window, lock cabi

In [ ]:
with open('result_list9.json', 'w') as file:
    json.dump(result_list9, file)

In [ ]:
result_list10 = run_model(splitted_lists['list_10'])

{'id': 'chatcmpl-8PgUquBEXSD5QapdWZz95Zo9mcglu', 'object': 'chat.completion', 'created': 1701132248, 'model': 'gpt-4-1106-vision-preview', 'usage': {'prompt_tokens': 5753, 'completion_tokens': 647, 'total_tokens': 6400}, 'choices': [{'message': {'role': 'assistant', 'content': 'Image 1: open door, close door, lock door, fasten door, block door, shut door, kick door, smash door, burst door, twist rope, roll paper, label box, arrange flowers, arrange chairs, shut window, open window, lock window, fasten window, block window, break window\n\nImage 2: arrange flowers, bind hair, twist hair, peel carrot, peel orange, peel apple, peel banana, slice apple, slice potato, slice meat, slice onion, carry umbrella, open umbrella, shut door, open door, close door, fasten door, lock door, label bottle, label box\n\nImage 3: pile books, arrange flowers, close book, open book, label bottle, label box, bind hair, twist hair, shut door, open door, close door, lock door, fasten window, arrange chairs, ca

In [112]:
with open('result_list10.json', 'w') as file:
    json.dump(result_list10, file)

In [203]:
result_list_oneshot_1to4 = result_list1 + result_list2 + result_list3 + result_list4
result_list_oneshot_5to8 = result_list5 + result_list6 + result_list7 + result_list8
result_list_oneshot_9to10 = result_list9 + result_list10

In [221]:
# Data Processing of the result
# final_result_oneshot_1to4 = []
final_result_oneshot_9to10 = []
for item in result_list_oneshot_9to10:
   if "Image 1:" in item and "Image 11:" in item:
    item = re.sub(r"Image 1:.*?Image 2:", "Image 2:", item, flags=re.DOTALL)
    if re.search(r"Image 11: (?:\w+ \w+, )*(?:\w+ \w+)\n", item):
      matches = re.findall(r"Image \d+: ([\w\s,]+?)(?=\n)", item)
    else:
      matches1 = re.findall(r"Image \d+: ([\w\s,]+?)(?=\n)", item)
      matches2 = re.findall(r"Image 11: ([\w\s,]+)", item)
      matches = matches1 + matches2
    print(len(matches))
   else:
    if re.search(r"Image 10: (?:\w+ \w+, )*(?:\w+ \w+)\n\n", item):
      matches = re.findall(r"Image \d+: ([\w\s,]+?)(?=\n)", item)
    else:
      matches1 = re.findall(r"Image \d+: ([\w\s,]+?)(?=\n)", item)
      if "Image 10:" in item:
        matches2 = re.findall(r"Image 10: ([\w\s,]+)", item)
      elif "Image 11:" in item:
        matches2 = re.findall(r"Image 11: ([\w\s,]+)", item)
      else:
        matches2 = []
      matches = matches1 + matches2
    print(len(matches))
   matches_flatten = []
   for pairs in matches:
    temp = pairs.split(', ')
    matches_flatten.append(temp)
   for data in matches_flatten:
    # final_result_oneshot_1to4.append(data)
    final_result_oneshot_9to10.append(data)
# len(final_result_oneshot_1to4)
len(final_result_oneshot_9to10)

10
10
10
10
10
10
10
10
10
10


100

In [206]:
final_result_oneshot_5to8 = []
for item in result_list_oneshot_5to8:
    matches = []
    # Check for Image 1 to Image 10
    if "Image 1:" in item:
        matches += re.findall(r"Image [1-9]: ([\w\s,]+?)(?=\n|$)", item)
        if "Image 10:" in item:
            matches += re.findall(r"Image 10: ([\w\s,]+?)(?=\n|$)", item)
    # If Image 1 is not present, check for Image 2 to Image 11
    else:
        matches += re.findall(r"Image [2-9]: ([\w\s,]+?)(?=\n|$)", item)
        if "Image 10:" in item:
            matches += re.findall(r"Image 10: ([\w\s,]+?)(?=\n|$)", item)
        if "Image 11:" in item:
            matches += re.findall(r"Image 11: ([\w\s,]+)(?=\n|$)", item)

    # Print the number of matches found
    print(len(matches))
    #print(matches)
    # Flatten the list of matches
    matches_flatten = [pairs.split(', ') for pairs in matches]

    # Extend the final result with the flattened matches
    final_result_oneshot_5to8.extend(matches)

group_size = 10

# Initialize an empty list to store the groups
groups = []

# Create 5 groups each containing 10 lists
for i in range(0, len(final_result_oneshot_5to8), group_size):
    group = final_result_oneshot_5to8[i:i + group_size]
    groups.append(group)

# Apply the condition to each group
for group in groups:
    first_list = group[0]
    first_prediction = first_list.split(', ')[0]  # Split the string and get the first prediction
    if first_prediction in ["pitch baseball", "throw baseball"]:
        # Remove the first list and replicate the last list in the group
        group.pop(0)
        group.append(group[-1])

# Check the groups after modification
modified_result_oneshot_5to8 = [item for group in groups for item in group]
len(modified_result_oneshot_5to8)

10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10


200

In [224]:
# Accuracy Top 20
final_result_one_shot = final_result_oneshot_1to4 + modified_result_oneshot_5to8 + final_result_oneshot_9to10
# final_result_one_shot = final_result_oneshot_1to4
correct_count = 0

for index, answer in enumerate(images_500_label):
    if answer in final_result_one_shot[index]:
        correct_count += 1
accuracy = correct_count / len(images_500_label)
print(f'Accuracy: {accuracy * 100:.2f}%')

Accuracy: 80.20%


In [229]:
# Accuracy Top 5
final_result_one_shot_top5 = [sublist[:5] for sublist in final_result_one_shot]
correct_count = 0
for index, answer in enumerate(images_500_label):
    if answer in final_result_one_shot_top5[index]:
        correct_count += 1
accuracy = correct_count / len(images_500_label)
print(f'Accuracy: {accuracy * 100:.2f}%')

Accuracy: 44.40%


In [226]:
# Accuracy Top 1
final_result_one_shot_top1 = [sublist[:1] for sublist in final_result_one_shot]
correct_count = 0
for index, answer in enumerate(images_500_label):
    if answer in final_result_one_shot_top1[index]:
        correct_count += 1
accuracy = correct_count / len(images_500_label)
print(f'Accuracy: {accuracy * 100:.2f}%')

Accuracy: 28.00%


### Zero shot

In [181]:
def get_openai_responses_zero_shots(api_key, image_files):

    image_data_list = []

    for image_file in image_files:
        buffer = BytesIO()
        image_file.save(buffer, format="JPEG")
        image_data = buffer.getvalue()
        base64_image = base64.b64encode(image_data).decode('utf-8')
        image_data_list.append({
            "type": "image_url",
            "image_url": {
                "url": f"data:image/jpeg;base64,{base64_image}"
            }
        })


    headers = {
        "Content-Type": "application/json",
        "Authorization": f"Bearer {api_key}"
    }

    # Prepare the payload for the request with multiple images
    payload = {
        "model": "gpt-4-vision-preview",  # Update this model name as per the latest API
        "messages": [
            {
                "role": "user",
                "content": [
                    {
                        "type": "text",
                        "text":  """ We are conducting an exercise to test the ability of an AI model to predict verb-noun pairs that describe the physical actions depicted in a series of images.\
                         We will upload a total of 10 images. \
                        Your tasks should satisfy the following 4 conditions":
                        1.For the 10 images, the task is to identify verb-noun pairs from given list that best describe the actions taking place in each image. \
                        2.The model should generate 20 possible verb-noun pairs from the given list for each image, resulting in a total of 200 pairs.\
                        3.10 possible verb-noun pairs for each image should be different. \
                        Each pair should be formatted like 'arrange chairs', and separated by a comma and without including line breaks and in the below format,
                        and the result must be like this format:
                         Image 1: bind hair, twist hair, label box, arrange flowers, arrange chairs, tie shoe, trim hair, carry bag, break glass, shave hair, \
                               bind hair, twist hair, label box, arrange flowers, arrange chairs, tie shoe, trim hair, carry bag, break glass, shave hair\
                         Image 2: like Image 1
                         ...
                         Image 10: like Image 1
                        4.The possible answers for the verb noun pairs are in this list: ['arrange chairs,arrange flowers,bake potato,beat eggs,\
                        bend knee,bend tree,bind hair,bite apple,block door,block window,boil egg,boil potato,break bowl,break cup,break door,break egg,break glass,\
                        break window,burn book,burn paper,burn tree,burn wood,burst balloon,burst door,carry bag,carry book,carry umbrella,chop carrot,chop meat,\
                        chop onion,chop tree,chop wood,close book,close cabinet,close door,close drawer,close window,coil rope,cook egg,cook meat,cook onion,\
                        cook potato,crack bottle,crack egg,crack glass,crack window,crash car,crop hair,cut apple,cut meat,cut onion,cut potato,cut tree,cut wood,\
                        fasten door,fasten window,fold paper,fry egg,fry meat,fry potato,grate carrot,grate potato,grind meat,hang bag,hang shirt,ignite paper,\
                        ignite wood,insert key,kick door,kick football,knot rope,label bottle,label box,lock cabinet,lock door,lock drawer,lock window,mash potato,\
                        mix eggs,open bottle,open box,open cabinet,open door,open drawer,open umbrella,open window,park car,peel apple,peel banana,peel carrot,\
                        peel orange,peel potato,pile books,pile boxes,pile wood,pitch baseball,ride bicycle,rip paper,roll paper,roll umbrella,saw tree,saw wood,\
                        scratch car,scratch knee,shave hair,shut door,shut window,skin knee,slice apple,slice meat,slice onion,slice potato,smash door,smash window,\
                        soak shirt,soak hair,spill coffee,split tree,split wood,squeeze orange,squeeze bottle,stain paper,stain shirt,stir coffee,stir soup,\
                        strip tree,tear book,tear shirt,tear paper,throw apple,throw baseball,throw football,throw frisbee,tie shoe,trim hair,trim tree,twist hair,\
                        twist rope,wrap book,wrap box']
                         """

                    },
                    *image_data_list
                ]
            }
        ],
        "max_tokens": 1000  #1000 is the best so far
    }

    # Send the request
    response = requests.post("https://api.openai.com/v1/chat/completions", headers=headers, json=payload).json()
    return response



In [182]:
# Run Chatgpt4
def run_model_zero_shot(image_list):
  responses500 = []
  # Replace with your actual API key
  api_key = "YOUR_OPENAI_KEY"

  for i in range(0, len(image_list), 10):
      images_10 = image_list[i:i+10]
      response_verb_noun = get_openai_responses_zero_shots(api_key, images_10)

      # Inspect the response
      print(response_verb_noun)

      try:
          content = response_verb_noun['choices'][0]['message']['content']
          responses500.append(content)
      except KeyError:
          print("KeyError encountered. Check the response structure.")
  return responses500

In [ ]:
result_zeroshot_list1 = run_model_zero_shot(splitted_lists['list_1'])

{'id': 'chatcmpl-8Pwe2jH8zaW7SWWw8SM7INGAYo6Fd', 'object': 'chat.completion', 'created': 1701194322, 'model': 'gpt-4-1106-vision-preview', 'usage': {'prompt_tokens': 7363, 'completion_tokens': 685, 'total_tokens': 8048}, 'choices': [{'message': {'role': 'assistant', 'content': 'Image 1: crash car, park car, scratch car, open door, close door, shut door, fasten door, lock door, open window, close window, crash car, park car, scratch car, open door, close door, shut door, fasten door, lock door, open window, close window\n\nImage 2: open window, close window, break window, smash window, fasten window, lock window, arrange chairs, arrange flowers, park car, open door, open window, close window, break window, smash window, fasten window, lock window, arrange chairs, arrange flowers, park car, open door\n\nImage 3: kick football, throw football, park car, ride bicycle, pitch baseball, peel banana, peel carrot, boil potato, bake potato, cut tree, kick football, throw football, park car, ride

In [ ]:
with open('result_zeroshot_list1.json', 'w') as file:
    json.dump(result_zeroshot_list1, file)

In [ ]:
result_zeroshot_list2 = run_model_zero_shot(splitted_lists['list_2'])

{'id': 'chatcmpl-8Pwg1ANVGfMNuvQqwJxdEGoPxWRRw', 'object': 'chat.completion', 'created': 1701194445, 'model': 'gpt-4-1106-vision-preview', 'usage': {'prompt_tokens': 6343, 'completion_tokens': 645, 'total_tokens': 6988}, 'choices': [{'message': {'role': 'assistant', 'content': 'Image 1: coil rope, tie shoe, wrap box, twist rope, close cabinet, fasten window, lock cabinet, bind hair, twist hair, pile boxes, coil rope, tie shoe, wrap box, twist rope, close cabinet, fasten window, lock cabinet, bind hair, twist hair, pile boxes\nImage 2: peel apple, slice apple, cut apple, chop apple, bite apple, arrange flowers, tear paper, label bottle, open bottle, throw apple, peel apple, slice apple, cut apple, chop apple, bite apple, arrange flowers, tear paper, label bottle, open bottle, throw apple\nImage 3: stir soup, cook meat, cook potato, mash potato, chop meat, slice meat, fry meat, boil potato, mix eggs, beat eggs, stir soup, cook meat, cook potato, mash potato, chop meat, slice meat, fry me

In [ ]:
with open('result_zeroshot_list2.json', 'w') as file:
    json.dump(result_zeroshot_list2, file)

In [ ]:
result_zeroshot_list3 = run_model_zero_shot(splitted_lists['list_3'])

{'id': 'chatcmpl-8Pwinf5qmadTAAj6zE1yCynfk99cD', 'object': 'chat.completion', 'created': 1701194617, 'model': 'gpt-4-1106-vision-preview', 'usage': {'prompt_tokens': 4813, 'completion_tokens': 641, 'total_tokens': 5454}, 'choices': [{'message': {'role': 'assistant', 'content': 'Image 1: stain shirt, arrange flowers, bind hair, burst balloon, carry bag, chop wood, close drawer, crack window, fold paper, fry potato, stain shirt, arrange flowers, bind hair, burst balloon, carry bag, chop wood, close drawer, crack window, fold paper, fry potato\n\nImage 2: pitch baseball, throw baseball, roll paper, arrange chairs, bend knee, burst balloon, carry book, chop wood, fry meat, hang shirt, pitch baseball, throw baseball, roll paper, arrange chairs, bend knee, burst balloon, carry book, chop wood, fry meat, hang shirt\n\nImage 3: arrange chairs, pile books, bind hair, boil egg, carry umbrella, chop meat, close door, cook potato, cut tree, fold paper, arrange chairs, pile books, bind hair, boil e

In [ ]:
with open('result_zeroshot_list3.json', 'w') as file:
    json.dump(result_zeroshot_list3, file)

In [ ]:
result_zeroshot_list4 = run_model_zero_shot(splitted_lists['list_4'])

{'id': 'chatcmpl-8PwtsOhbiBFCYsD4HNhHGZInJ3Mpv', 'object': 'chat.completion', 'created': 1701195304, 'model': 'gpt-4-1106-vision-preview', 'usage': {'prompt_tokens': 4813, 'completion_tokens': 697, 'total_tokens': 5510}, 'choices': [{'message': {'role': 'assistant', 'content': 'Image 1: tear paper, rip paper, open window, label box, pile books, cut tree, burst balloon, block window, arrange flowers, arrange chairs, peel banana, break glass, bend tree, burn book, peel orange, roll paper, shut door, chop wood, fry meat, mash potato\n\nImage 2: chop onion, slice onion, peel potato, cut potato, cook onion, fry potato, dice onion, grate carrot, chop meat, cut onion, peel carrot, chop wood, mix eggs, bend tree, peel banana, carry bag, crack egg, bind hair, break egg, open drawer\n\nImage 3: chop carrot, cook meat, peel potato, open drawer, stir soup, fry meat, grate carrot, cut meat, slice meat, slice onion, mix eggs, chop onion, peel apple, cut potato, chop wood, dice onion, mash potato, co

In [ ]:
with open('result_zeroshot_list4.json', 'w') as file:
    json.dump(result_zeroshot_list4, file)

In [ ]:
result_zeroshot_list5 = run_model_zero_shot(splitted_lists['list_5'])

{'id': 'chatcmpl-8Pww39aRIhLvkIW8vbFBRQ5DYzCxd', 'object': 'chat.completion', 'created': 1701195439, 'model': 'gpt-4-1106-vision-preview', 'usage': {'prompt_tokens': 6853, 'completion_tokens': 644, 'total_tokens': 7497}, 'choices': [{'message': {'role': 'assistant', 'content': 'Image 1: bite apple, peel apple, slice apple, throw apple, cut apple, mash potato, boil potato, fry potato, chop carrot, grate carrot, peel banana, peel orange, squeeze orange, open bottle, close bottle, label bottle, burst balloon, crack bottle, roll paper, tie shoe\n\nImage 2: bind hair, twist hair, trim hair, shave hair, soak hair, arrange flowers, peel banana, squeeze orange, label bottle, burst balloon, close door, close drawer, carry bag, tie shoe, shut door, shut window, fasten door, twist rope, scratch knee, skin knee\n\nImage 3: bake potato, mash potato, fry potato, chop potato, cut potato, grind meat, boil potato, peel potato, peel carrot, grate potato, stir soup, cook potato, arrange flowers, carry ba

In [ ]:
with open('result_zeroshot_list5.json', 'w') as file:
    json.dump(result_zeroshot_list5, file)

In [ ]:
result_zeroshot_list6 = run_model_zero_shot(splitted_lists['list_6'])

{'id': 'chatcmpl-8PwySegPi889ACHLRbbiu8gT4Qt9t', 'object': 'chat.completion', 'created': 1701195588, 'model': 'gpt-4-1106-vision-preview', 'usage': {'prompt_tokens': 5833, 'completion_tokens': 649, 'total_tokens': 6482}, 'choices': [{'message': {'role': 'assistant', 'content': 'Image 1: close window, lock window, fasten window, open window, block window, smash window, crack window, arrange chairs, bind hair, twist hair, close window, lock window, fasten window, open window, block window, smash window, crack window, arrange flowers, boil egg, fry meat\n\nImage 2: open drawer, close drawer, lock drawer, close cabinet, open cabinet, arrange chairs, label box, pile books, arrange flowers, tie shoe, open drawer, close drawer, lock drawer, close cabinet, open cabinet, peel orange, pile boxes, carry book, bind hair, twist hair\n\nImage 3: coil rope, twist rope, knot rope, arrange chairs, arrange flowers, lock door, close door, open door, pile wood, carry bag, coil rope, twist rope, knot rope,

In [ ]:
with open('result_zeroshot_list6.json', 'w') as file:
    json.dump(result_zeroshot_list6, file)

In [ ]:
result_zeroshot_list7 = run_model_zero_shot(splitted_lists['list_7'])

{'id': 'chatcmpl-8Px41VNucLYUdgxPVeiblubbzKG4u', 'object': 'chat.completion', 'created': 1701195933, 'model': 'gpt-4-1106-vision-preview', 'usage': {'prompt_tokens': 5493, 'completion_tokens': 645, 'total_tokens': 6138}, 'choices': [{'message': {'role': 'assistant', 'content': 'Image 1: fry egg, cook egg, break egg, crack egg, peel apple, cut apple, open bottle, fry potato, bake potato, boil potato, fry egg, cook egg, break egg, crack egg, peel apple, cut apple, open bottle, fry potato, bake potato, boil potato\n\nImage 2: carry bag, hang bag, tie shoe, peel banana, peel carrot, peel orange, close drawer, lock drawer, open drawer, fold paper, carry bag, hang bag, tie shoe, peel banana, peel carrot, peel orange, close drawer, lock drawer, open drawer, fold paper\n\nImage 3: slice apple, cut apple, peel apple, bite apple, throw apple, arrange flowers, pile books, ignite paper, burn paper, tie shoe, slice apple, cut apple, peel apple, bite apple, throw apple, arrange flowers, pile books, 

In [ ]:
with open('result_zeroshot_list7.json', 'w') as file:
    json.dump(result_zeroshot_list7, file)

In [ ]:
result_zeroshot_list8 = run_model_zero_shot(splitted_lists['list_8'])

{'id': 'chatcmpl-8Px6STOComugKKtxdoENPiHlSQV0I', 'object': 'chat.completion', 'created': 1701196084, 'model': 'gpt-4-1106-vision-preview', 'usage': {'prompt_tokens': 6173, 'completion_tokens': 705, 'total_tokens': 6878}, 'choices': [{'message': {'role': 'assistant', 'content': "Based on the instructions and the images provided, here are my verb-noun pair predictions for each image:\n\nImage 1: arrange chairs, arrange chairs, arrange chairs, arrange chairs, arrange chairs, arrange chairs, arrange chairs, arrange chairs, arrange chairs, arrange chairs, block door, block door, block door, block door, block door, block door, block door, block door, block door, block door\nImage 2: spill coffee, spill coffee, spill coffee, spill coffee, spill coffee, spill coffee, spill coffee, spill coffee, spill coffee, spill coffee, break cup, break cup, break cup, break cup, break cup, break cup, break cup, break cup, break cup, break cup\nImage 3: slice apple, slice apple, slice apple, slice apple, sli

In [ ]:
with open('result_zeroshot_list8.json', 'w') as file:
    json.dump(result_zeroshot_list8, file)

In [120]:
result_zeroshot_list9 = run_model_zero_shot(splitted_lists['list_9'])

{'id': 'chatcmpl-8Py15SdffBPzhaHftWPcx74JK6ZlK', 'object': 'chat.completion', 'created': 1701199595, 'model': 'gpt-4-1106-vision-preview', 'usage': {'prompt_tokens': 6343, 'completion_tokens': 643, 'total_tokens': 6986}, 'choices': [{'message': {'role': 'assistant', 'content': 'Image 1: crack glass, break glass, smash window, crack window, break window, smash door, break door, smash door, crack window, break window, crack glass, break glass, smash window, crack window, break window, smash door, break door, smash door, crack window, break window\nImage 2: whip cream, mash potato, beat eggs, stir soup, mix eggs, grate potato, stir coffee, mash potato, whip cream, stir soup, whip cream, mash potato, beat eggs, stir soup, mix eggs, grate potato, stir coffee, mash potato, whip cream, stir soup\nImage 3: wrap book, open box, label box, pile books, arrange chairs, fold paper, pile boxes, open bottle, label bottle, wrap box, wrap book, open box, label box, pile books, arrange chairs, fold pape

In [ ]:
with open('result_zeroshot_list9.json', 'w') as file:
    json.dump(result_zeroshot_list9, file)

In [ ]:
result_zeroshot_list10 = run_model_zero_shot(splitted_lists['list_10'])

{'id': 'chatcmpl-8PxF0auVQ4wtqLxznmnEJU7BWqZRl', 'object': 'chat.completion', 'created': 1701196614, 'model': 'gpt-4-1106-vision-preview', 'usage': {'prompt_tokens': 5323, 'completion_tokens': 651, 'total_tokens': 5974}, 'choices': [{'message': {'role': 'assistant', 'content': 'Image 1: close door, open door, lock door, lock cabinet, fasten door, shut door, open cabinet, close cabinet, open drawer, close drawer, close door, open door, lock door, lock cabinet, fasten door, shut door, open cabinet, close cabinet, open drawer, close drawer\nImage 2: arrange flowers, hold flowers, carry bag, carry umbrella, wrap box, label box, open box, close drawer, tie shoe, bend knee, arrange flowers, hold flowers, carry bag, carry umbrella, wrap box, label box, open box, close drawer, tie shoe, bend knee\nImage 3: pile books, arrange chairs, label box, stack boxes, carry book, open book, close book, tear book, wrap book, tie shoe, pile books, arrange chairs, label box, stack boxes, carry book, open bo

In [ ]:
with open('result_zeroshot_list10.json', 'w') as file:
    json.dump(result_zeroshot_list10, file)

In [ ]:
print(result_zeroshot_list5[0])

Image 1: bite apple, peel apple, slice apple, throw apple, cut apple, mash potato, boil potato, fry potato, chop carrot, grate carrot, peel banana, peel orange, squeeze orange, open bottle, close bottle, label bottle, burst balloon, crack bottle, roll paper, tie shoe

Image 2: bind hair, twist hair, trim hair, shave hair, soak hair, arrange flowers, peel banana, squeeze orange, label bottle, burst balloon, close door, close drawer, carry bag, tie shoe, shut door, shut window, fasten door, twist rope, scratch knee, skin knee

Image 3: bake potato, mash potato, fry potato, chop potato, cut potato, grind meat, boil potato, peel potato, peel carrot, grate potato, stir soup, cook potato, arrange flowers, carry bag, roll paper, open box, label box, close cabinet, twist rope, cut wood

Image 4: open bottle, close bottle, label bottle, carry bag, crack bottle, roll paper, peel banana, peel orange, squeeze orange, fill glass, pour water, burst balloon, open window, close window, lock cabinet, t

In [184]:
# Data Processing of the result
final_result = []
for item in result_zeroshot_list9:
   if "Image 1:" in item and "Image 11:" in item:
    item = re.sub(r"Image 1:.*?Image 2:", "Image 2:", item, flags=re.DOTALL)
    if re.search(r"Image 11: (?:\w+ \w+, )*(?:\w+ \w+)\n", item):
      matches = re.findall(r"Image \d+: ([\w\s,]+?)(?=\n)", item)
    else:
      matches1 = re.findall(r"Image \d+: ([\w\s,]+?)(?=\n)", item)
      matches2 = re.findall(r"Image 11: ([\w\s,]+)", item)
      matches = matches1 + matches2
    print(len(matches))
   else:
    if re.search(r"Image 10: (?:\w+ \w+, )*(?:\w+ \w+)\n\n", item):
      matches = re.findall(r"Image \d+: ([\w\s,]+?)(?=\n)", item)
    else:
      matches1 = re.findall(r"Image \d+: ([\w\s,]+?)(?=\n)", item)
      if "Image 10:" in item:
        matches2 = re.findall(r"Image 10: ([\w\s,]+)", item)
      elif "Image 11:" in item:
        matches2 = re.findall(r"Image 11: ([\w\s,]+)", item)
      else:
        matches2 = []
      matches = matches1 + matches2
    print(len(matches))
   matches_flatten = []
   for pairs in matches:
    temp = pairs.split(', ')
    matches_flatten.append(temp)
   for data in matches_flatten:
    final_result.append(data)
len(final_result)

10
10
10
10
10


50

In [146]:
all_positive_images = []
for i in range(len(positive_images)):
    label = verb_noun[i]
    for image in positive_images[i]:
        all_positive_images.append((label, image))
sample_positive_images = []
for image, label in zip(images_500, images_500_label):
  sample_positive_images.append((label, image))

remaining_images = []
for item in all_positive_images:
  if item not in sample_positive_images:
    remaining_images.append(item)
len(remaining_images)


1540

In [151]:
sampled_images_50 = random.sample(remaining_images, 50)
newimages_50_forlist9 = [item[1] for item in sampled_images_50]
newanswers_50_forlist9 = [item[0] for item in sampled_images_50]
splitted_lists['list_9'] = newimages_50_forlist9

In [155]:
images_500_label_new = images_500_label[0:400] + newanswers_50_forlist9 + images_500_label[-50:]

In [183]:
result_zeroshot_list9 = run_model_zero_shot(splitted_lists['list_9'])

{'id': 'chatcmpl-8PycMYerF23pb0IEguVotKQnBygt4', 'object': 'chat.completion', 'created': 1701201906, 'model': 'gpt-4-1106-vision-preview', 'usage': {'prompt_tokens': 5843, 'completion_tokens': 643, 'total_tokens': 6486}, 'choices': [{'message': {'role': 'assistant', 'content': 'Image 1: carry umbrella, open umbrella, carry bag, open door, open window, bend knee, bite apple, peel orange, arrange flowers, arrange chairs, close umbrella, lock door, park car, ride bicycle, tear paper, twist hair, bind hair, wear coat, fasten door, shut window\nImage 2: squeeze orange, peel orange, tear paper, peel banana, peel apple, peel carrot, slice potato, mash potato, cut potato, grate potato, boil potato, bite apple, carry bag, bend knee, chop wood, chop meat, chop carrot, fry potato, cook onion, open bottle\nImage 3: break glass, crack glass, smash door, cut wood, cut onion, cut potato, cut apple, break bowl, break cup, break window, crack bottle, crack egg, chop wood, chop onion, chop meat, chop ca

In [185]:
with open('result_zeroshot_list9.json', 'w') as file:
    json.dump(result_zeroshot_list9, file)

In [188]:
### Get the result from zero shot
result_zeroshot_list = result_zeroshot_list1 + result_zeroshot_list2 + result_zeroshot_list3 + result_zeroshot_list4 + result_zeroshot_list5 + \
             result_zeroshot_list6 + result_zeroshot_list7 + result_zeroshot_list8 + result_zeroshot_list9 + result_zeroshot_list10
# Data Processing of the result
final_result_zeroshot = []
for item in result_zeroshot_list:
   if "Image 1:" in item and "Image 11:" in item:
    item = re.sub(r"Image 1:.*?Image 2:", "Image 2:", item, flags=re.DOTALL)
    if re.search(r"Image 11: (?:\w+ \w+, )*(?:\w+ \w+)\n", item):
      matches = re.findall(r"Image \d+: ([\w\s,]+?)(?=\n)", item)
    else:
      matches1 = re.findall(r"Image \d+: ([\w\s,]+?)(?=\n)", item)
      matches2 = re.findall(r"Image 11: ([\w\s,]+)", item)
      matches = matches1 + matches2
   else:
    if re.search(r"Image 10: (?:\w+ \w+, )*(?:\w+ \w+)\n\n", item):
      matches = re.findall(r"Image \d+: ([\w\s,]+?)(?=\n)", item)
    else:
      matches1 = re.findall(r"Image \d+: ([\w\s,]+?)(?=\n)", item)
      if "Image 10:" in item:
        matches2 = re.findall(r"Image 10: ([\w\s,]+)", item)
      elif "Image 11:" in item:
        matches2 = re.findall(r"Image 11: ([\w\s,]+)", item)
      else:
        matches2 = []
      matches = matches1 + matches2
   matches_flatten = []
   for pairs in matches:
    temp = pairs.split(', ')
    matches_flatten.append(temp)
   for data in matches_flatten:
    final_result_zeroshot.append(data)
len(final_result_zeroshot)

500

In [230]:
# Accuracy Top 20
correct_count = 0
for index, answer in enumerate(images_500_label_new):
    if answer in final_result_zeroshot[index]:
        correct_count += 1
accuracy = correct_count / len(images_500_label_new)
print(f'Accuracy: {accuracy * 100:.3f}%')

Accuracy: 87.000%


In [231]:
# Accuracy Top 5
final_result_top5_zeroshot = [sublist[:5] for sublist in final_result_zeroshot]
correct_count = 0
for index, answer in enumerate(images_500_label_new):
    if answer in final_result_top5_zeroshot[index]:
        correct_count += 1
accuracy = correct_count / len(images_500_label_new)
print(f'Accuracy: {accuracy * 100:.3f}%')

Accuracy: 79.600%


In [232]:
# Accuracy Top 1
final_result_top1_zeroshot = [sublist[:1] for sublist in final_result_zeroshot]
correct_count = 0
for index, answer in enumerate(images_500_label_new):
    if answer in final_result_top1_zeroshot[index]:
        correct_count += 1
accuracy = correct_count / len(images_500_label_new)
print(f'Accuracy: {accuracy * 100:.3f}%')

Accuracy: 53.800%


In [195]:
import json
filepath = '/content/result_list1.json'
with open(filepath, 'r') as file:
    result_list1 = json.load(file)
filepath = '/content/result_list2.json'
with open(filepath, 'r') as file:
    result_list2 = json.load(file)
filepath = '/content/result_list3.json'
with open(filepath, 'r') as file:
    result_list3 = json.load(file)
filepath = '/content/result_list4.json'
with open(filepath, 'r') as file:
    result_list4 = json.load(file)

filepath = '/content/result_list5.json'
with open(filepath, 'r') as file:
    result_list5 = json.load(file)
filepath = '/content/result_list6.json'
with open(filepath, 'r') as file:
    result_list6 = json.load(file)
filepath = '/content/result_list7.json'
with open(filepath, 'r') as file:
    result_list7 = json.load(file)
filepath = '/content/result_list8.json'
with open(filepath, 'r') as file:
    result_list8 = json.load(file)

filepath = '/content/result_list9.json'
with open(filepath, 'r') as file:
    result_list9 = json.load(file)
filepath = '/content/result_list10.json'
with open(filepath, 'r') as file:
    result_list10 = json.load(file)

In [202]:
import re


final_result = []

for item in result_list5:
    matches = []
    # Check for Image 1 to Image 10
    if "Image 1:" in item:
        matches += re.findall(r"Image [1-9]: ([\w\s,]+?)(?=\n|$)", item)
        if "Image 10:" in item:
            matches += re.findall(r"Image 10: ([\w\s,]+?)(?=\n|$)", item)
    # If Image 1 is not present, check for Image 2 to Image 11
    else:
        matches += re.findall(r"Image [2-9]: ([\w\s,]+?)(?=\n|$)", item)
        if "Image 10:" in item:
            matches += re.findall(r"Image 10: ([\w\s,]+?)(?=\n|$)", item)
        if "Image 11:" in item:
            matches += re.findall(r"Image 11: ([\w\s,]+)(?=\n|$)", item)

    # Print the number of matches found
    print(len(matches))
    #print(matches)
    # Flatten the list of matches
    matches_flatten = [pairs.split(', ') for pairs in matches]

    # Extend the final result with the flattened matches
    final_result.extend(matches)

group_size = 10

# Initialize an empty list to store the groups
groups = []

# Create 5 groups each containing 10 lists
for i in range(0, len(final_result), group_size):
    group = final_result[i:i + group_size]
    groups.append(group)

# Apply the condition to each group
for group in groups:
    first_list = group[0]
    first_prediction = first_list.split(', ')[0]  # Split the string and get the first prediction
    if first_prediction in ["pitch baseball", "throw baseball"]:
        # Remove the first list and replicate the last list in the group
        group.pop(0)
        group.append(group[-1])

# Check the groups after modification
modified_result = [item for group in groups for item in group]
len(modified_result)

10
10
10
10
10


50